# Regresión Lineal con Múltiples Variables

## Objetivos de Aprendizaje
En este notebook aprenderás:
- Extender regresión lineal a múltiples features
- Implementar vectorización con NumPy para eficiencia
- Desarrollar gradient descent para múltiples variables
- Aplicar el modelo a predicción de precios de casas

## Problema de Negocio
**Escenario**: Eres un agente inmobiliario que necesita estimar precios de casas basándose en múltiples características:
- **Tamaño** de la casa (pies cuadrados)
- **Número de habitaciones**
- **Número de pisos**
- **Edad** de la casa (años)

**Objetivo**: Crear un modelo que prediga el precio usando todas estas características simultáneamente.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math

# Configuración
plt.style.use('default')
np.set_printoptions(precision=2)

print("Librerías importadas correctamente")
print("Listo para regresión multivariable")

## 1. Dataset: Precios de Casas

### 1.1 Cargar y Explorar los Datos

In [ ]:
# Función para cargar datos de casas
def cargar_datos_casas():
    """
    Carga dataset de precios de casas con múltiples features
    
    Returns:
        X_train: Matriz de features (m, n)
        y_train: Vector de precios (m,)
        feature_names: Nombres de las características
    """
    # Dataset real de casas
    # Cada fila: [tamaño_sqft, habitaciones, pisos, edad]
    X_train = np.array([
        [2104, 5, 1, 45],    # Casa 1
        [1416, 3, 2, 40],    # Casa 2  
        [852,  2, 1, 35],    # Casa 3
        [1940, 4, 1, 10],    # Casa 4
        [1539, 3, 2, 25],    # Casa 5
        [3000, 4, 2, 8],     # Casa 6
        [1230, 3, 1, 15],    # Casa 7
        [2145, 4, 1, 12],    # Casa 8
        [1736, 3, 2, 30],    # Casa 9
        [1000, 2, 1, 50]     # Casa 10
    ])
    
    # Precios correspondientes (en miles de dólares)
    y_train = np.array([460, 232, 178, 500, 315, 740, 285, 510, 390, 180])
    
    # Nombres de features
    feature_names = ['Tamaño (sqft)', 'Habitaciones', 'Pisos', 'Edad (años)']
    
    return X_train, y_train, feature_names

# Cargar datos
X_train, y_train, feature_names = cargar_datos_casas()

print(f"Dataset de casas cargado:")
print(f"  • Número de casas (m): {X_train.shape[0]}")
print(f"  • Número de features (n): {X_train.shape[1]}")
print(f"  • Shape de X_train: {X_train.shape}")
print(f"  • Shape de y_train: {y_train.shape}")

print(f"\nPrimeras 5 casas:")
print(f"{'':>5} {'Tamaño':>8} {'Habitaciones':>12} {'Pisos':>6} {'Edad':>5} {'Precio':>8}")
print("-" * 50)
for i in range(5):
    print(f"Casa {i+1:1d}: {X_train[i,0]:>6.0f} {X_train[i,1]:>10.0f} {X_train[i,2]:>8.0f} {X_train[i,3]:>6.0f} ${y_train[i]:>6.0f}k")

### 1.2 Visualización del Dataset

In [ ]:
# Visualizar cada feature vs precio
plt.figure(figsize=(16, 4))

for i in range(len(feature_names)):
    plt.subplot(1, 4, i+1)
    plt.scatter(X_train[:, i], y_train, alpha=0.7, s=50)
    plt.xlabel(feature_names[i])
    plt.ylabel('Precio (miles $)' if i == 0 else '')
    plt.title(f'{feature_names[i]} vs Precio')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Observaciones:")
print("  • Tamaño: Relación positiva clara - casas más grandes cuestan más")
print("  • Habitaciones: Tendencia positiva - más habitaciones, mayor precio")
print("  • Pisos: Relación menos clara, pero casas de 2 pisos tienden a costar más")
print("  • Edad: Relación negativa - casas más nuevas cuestan más")

## 2. Modelo de Regresión Multivariable

### 2.1 Extensión del Modelo Lineal

**Modelo con una variable**: $f_{w,b}(x) = wx + b$

**Modelo con múltiples variables**: 
$$f_{\mathbf{w},b}(\mathbf{x}) = w_0x_0 + w_1x_1 + w_2x_2 + w_3x_3 + b$$

**En notación vectorial**:
$$f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b$$

Donde:
- **x**: Vector de features [tamaño, habitaciones, pisos, edad]
- **w**: Vector de pesos [w₀, w₁, w₂, w₃]
- **b**: Término de sesgo (bias)
- **·**: Producto punto

In [ ]:
# Implementación de predicción multivariable

def predecir_una_casa_loop(x, w, b):
    """
    Predicción para una casa usando loop (versión educativa)
    
    Args:
        x (ndarray): Vector de features para una casa (n,)
        w (ndarray): Vector de pesos (n,)
        b (scalar): Bias
    
    Returns:
        prediction (scalar): Precio predicho
    """
    n = x.shape[0]
    prediccion = 0
    
    # Sumar cada w_i * x_i
    for i in range(n):
        prediccion += w[i] * x[i]
    
    # Añadir bias
    prediccion += b
    
    return prediccion

def predecir_una_casa_vectorizada(x, w, b):
    """
    Predicción vectorizada (versión eficiente)
    
    Args:
        x (ndarray): Vector de features (n,)
        w (ndarray): Vector de pesos (n,)
        b (scalar): Bias
        
    Returns:
        prediction (scalar): Precio predicho
    """
    return np.dot(x, w) + b

# Probar ambas implementaciones
# Parámetros ejemplo (valores cercanos a óptimos)
w_inicial = np.array([0.39, 18.75, -53.36, -26.42])
b_inicial = 785.18

# Tomar primera casa como ejemplo
x_ejemplo = X_train[0]  # [2104, 5, 1, 45]
precio_real = y_train[0]  # 460

print(f"Casa ejemplo: {x_ejemplo}")
print(f"Precio real: ${precio_real}k")
print(f"\nPredicciones:")

# Predicción con loop
pred_loop = predecir_una_casa_loop(x_ejemplo, w_inicial, b_inicial)
print(f"  Con loop: ${pred_loop:.1f}k")

# Predicción vectorizada
pred_vectorizada = predecir_una_casa_vectorizada(x_ejemplo, w_inicial, b_inicial)
print(f"  Vectorizada: ${pred_vectorizada:.1f}k")
print(f"  ¿Son iguales? {np.isclose(pred_loop, pred_vectorizada)}")

print(f"\nDesglose de la predicción vectorizada:")
for i, (feature, peso, valor) in enumerate(zip(feature_names, w_inicial, x_ejemplo)):
    contribucion = peso * valor
    print(f"  {feature}: {peso:.2f} × {valor} = {contribucion:.1f}")
print(f"  Bias: {b_inicial:.1f}")
print(f"  Total: {pred_vectorizada:.1f}")

### 2.2 Comparación de Velocidad: Loop vs Vectorización

In [ ]:
import time

# Crear dataset grande para comparar velocidad
np.random.seed(42)
n_casas_grandes = 100000
X_grande = np.random.rand(n_casas_grandes, 4) * 1000  # Features aleatorias
w_test = np.array([0.5, 10, -20, -15])
b_test = 100

print(f"Comparación de velocidad con {n_casas_grandes:,} casas:")
print("-" * 50)

# Método con loop
start_time = time.time()
predicciones_loop = []
for i in range(n_casas_grandes):
    pred = predecir_una_casa_loop(X_grande[i], w_test, b_test)
    predicciones_loop.append(pred)
tiempo_loop = time.time() - start_time

# Método vectorizado
start_time = time.time()
predicciones_vectorizadas = X_grande @ w_test + b_test  # @ es equivalente a np.dot
tiempo_vectorizado = time.time() - start_time

# Verificar que son iguales
predicciones_loop = np.array(predicciones_loop)
son_iguales = np.allclose(predicciones_loop, predicciones_vectorizadas)

print(f"Tiempo con loop:      {tiempo_loop:.4f} segundos")
print(f"Tiempo vectorizado:   {tiempo_vectorizado:.4f} segundos")
print(f"Speedup:              {tiempo_loop/tiempo_vectorizado:.1f}x más rápido")
print(f"Resultados iguales:   {son_iguales}")

print(f"\nConclusión: La vectorización es {tiempo_loop/tiempo_vectorizado:.0f}x más rápida")
print("Por eso siempre usaremos NumPy en machine learning")

# Limpiar memoria
del X_grande, predicciones_loop, predicciones_vectorizadas

## 3. Función de Costo para Múltiples Variables

La función de costo se extiende naturalmente:

$$J(\mathbf{w},b) = \frac{1}{2m} \sum_{i=0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2$$

Donde $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b$

In [ ]:
# Implementación de función de costo multivariable

def calcular_costo_multivariable(X, y, w, b):
    """
    Calcula la función de costo para regresión lineal multivariable
    
    Args:
        X (ndarray): Matriz de features (m, n)
        y (ndarray): Vector de targets (m,)
        w (ndarray): Vector de pesos (n,)
        b (scalar): Bias
        
    Returns:
        costo (scalar): Valor de la función de costo
    """
    m = X.shape[0]
    costo = 0.0
    
    for i in range(m):
        # Predicción para la casa i usando producto punto
        f_wb_i = np.dot(X[i], w) + b
        
        # Error cuadrático
        costo += (f_wb_i - y[i]) ** 2
    
    costo = costo / (2 * m)
    return costo

# Probar función de costo
costo_inicial = calcular_costo_multivariable(X_train, y_train, w_inicial, b_inicial)
print(f"Costo con parámetros iniciales: {costo_inicial:.3f}")

# Probar con diferentes parámetros
parametros_prueba = [
    (np.zeros(4), 0, "Ceros"),
    (np.ones(4), 0, "Unos"),
    (w_inicial, b_inicial, "Inicial (cerca del óptimo)"),
    (np.array([1, 0, 0, 0]), 100, "Solo tamaño")
]

print(f"\nComparación de diferentes parámetros:")
print(f"{'Descripción':<25} {'Costo':<10}")
print("-" * 35)

for w_test, b_test, desc in parametros_prueba:
    costo = calcular_costo_multivariable(X_train, y_train, w_test, b_test)
    print(f"{desc:<25} {costo:<10.3f}")

print(f"\nMenor costo indica mejor ajuste a los datos")

## 4. Gradient Descent para Múltiples Variables

### 4.1 Gradientes (Derivadas Parciales)

Para múltiples variables, necesitamos calcular gradientes para cada parámetro:

$$\frac{\partial J(\mathbf{w},b)}{\partial w_j} = \frac{1}{m} \sum_{i=0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_j^{(i)}$$

$$\frac{\partial J(\mathbf{w},b)}{\partial b} = \frac{1}{m} \sum_{i=0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})$$

**Algoritmo de actualización**:
$$w_j = w_j - \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j}$$
$$b = b - \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}$$

In [ ]:
# Implementación de gradientes multivariables

def calcular_gradientes_multivariable(X, y, w, b):
    """
    Calcula gradientes para regresión lineal multivariable
    
    Args:
        X (ndarray): Matriz de features (m, n)
        y (ndarray): Vector de targets (m,)
        w (ndarray): Vector de pesos (n,)
        b (scalar): Bias
        
    Returns:
        dj_dw (ndarray): Gradientes respecto a w (n,)
        dj_db (scalar): Gradiente respecto a b
    """
    m, n = X.shape  # m ejemplos, n features
    
    # Inicializar gradientes
    dj_dw = np.zeros((n,))
    dj_db = 0.
    
    for i in range(m):
        # Error para el ejemplo i
        error = (np.dot(X[i], w) + b) - y[i]
        
        # Gradientes para cada w_j
        for j in range(n):
            dj_dw[j] += error * X[i, j]
        
        # Gradiente para b
        dj_db += error
    
    # Promediar gradientes
    dj_dw = dj_dw / m
    dj_db = dj_db / m
    
    return dj_dw, dj_db

# Probar cálculo de gradientes
dj_dw_test, dj_db_test = calcular_gradientes_multivariable(X_train, y_train, w_inicial, b_inicial)

print(f"Gradientes con parámetros iniciales:")
print(f"dJ/db: {dj_db_test:.6f}")
print(f"dJ/dw: {dj_dw_test}")

print(f"\nInterpretación de gradientes:")
for i, (feature, grad) in enumerate(zip(feature_names, dj_dw_test)):
    direccion = "aumentar" if grad < 0 else "disminuir"
    print(f"  {feature}: {grad:.6f} → {direccion} w[{i}]")

direccion_b = "aumentar" if dj_db_test < 0 else "disminuir"
print(f"  Bias: {dj_db_test:.6f} → {direccion_b} b")

### 4.2 Implementación Completa de Gradient Descent

In [ ]:
# Gradient descent completo para múltiples variables

def gradient_descent_multivariable(X, y, w_inicial, b_inicial, 
                                  calcular_costo_func, calcular_gradientes_func,
                                  alpha, num_iteraciones):
    """
    Implementa gradient descent para regresión lineal multivariable
    
    Args:
        X (ndarray): Matriz de features (m, n)
        y (ndarray): Vector de targets (m,)
        w_inicial (ndarray): Valores iniciales de w (n,)
        b_inicial (scalar): Valor inicial de b
        calcular_costo_func: Función para calcular costo
        calcular_gradientes_func: Función para calcular gradientes
        alpha (float): Learning rate
        num_iteraciones (int): Número de iteraciones
        
    Returns:
        w (ndarray): Parámetros w optimizados
        b (scalar): Parámetro b optimizado
        historial_costos (list): Historia del costo
    """
    # Inicializar parámetros (evitar modificar originales)
    w = copy.deepcopy(w_inicial)
    b = b_inicial
    
    # Historia para tracking
    historial_costos = []
    
    for i in range(num_iteraciones):
        # Calcular gradientes
        dj_dw, dj_db = calcular_gradientes_func(X, y, w, b)
        
        # Actualizar parámetros
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        
        # Guardar costo
        if i < 100000:  # Evitar usar mucha memoria
            costo = calcular_costo_func(X, y, w, b)
            historial_costos.append(costo)
        
        # Imprimir progreso
        if i % math.ceil(num_iteraciones / 10) == 0:
            costo_actual = calcular_costo_func(X, y, w, b)
            print(f"Iteración {i:4d}: Costo = {costo_actual:8.2f}")
    
    return w, b, historial_costos

# Ejecutar gradient descent
print("Entrenando modelo de regresión multivariable...")
print("=" * 50)

# Parámetros de entrenamiento
w_inicial = np.zeros(4)  # Empezar desde cero
b_inicial = 0.
learning_rate = 5.0e-7  # Learning rate pequeño para datos no normalizados
iteraciones = 1000

# Entrenar modelo
w_final, b_final, hist_costos = gradient_descent_multivariable(
    X_train, y_train, w_inicial, b_inicial,
    calcular_costo_multivariable, calcular_gradientes_multivariable,
    learning_rate, iteraciones
)

print(f"\nEntrenamiento completado!")
print(f"Parámetros finales:")
for i, (feature, peso) in enumerate(zip(feature_names, w_final)):
    print(f"  w[{i}] ({feature}): {peso:.4f}")
print(f"  b (bias): {b_final:.4f}")
print(f"Costo final: {hist_costos[-1]:.2f}")

### 4.3 Visualización del Entrenamiento

In [ ]:
# Visualizar proceso de entrenamiento
plt.figure(figsize=(12, 5))

# Subplot 1: Evolución del costo
plt.subplot(1, 2, 1)
plt.plot(hist_costos, 'b-', linewidth=2)
plt.title('Evolución del Costo Durante el Entrenamiento')
plt.xlabel('Iteraciones')
plt.ylabel('Costo J(w,b)')
plt.grid(True, alpha=0.3)

# Marcar costo final
plt.plot(len(hist_costos)-1, hist_costos[-1], 'ro', markersize=8)
plt.annotate(f'Final: {hist_costos[-1]:.1f}', 
             xy=(len(hist_costos)-1, hist_costos[-1]),
             xytext=(len(hist_costos)*0.6, hist_costos[-1]*1.2),
             arrowprops=dict(arrowstyle='->', color='red'))

# Subplot 2: Parámetros finales
plt.subplot(1, 2, 2)
colores = ['skyblue', 'lightgreen', 'lightcoral', 'lightsalmon']
barras = plt.bar(range(len(feature_names)), w_final, color=colores, alpha=0.7, edgecolor='black')
plt.title('Pesos Finales por Feature')
plt.xlabel('Features')
plt.ylabel('Peso w_j')
plt.xticks(range(len(feature_names)), [f.replace(' ', '\n').replace('(', '\n(') for f in feature_names])
plt.grid(True, alpha=0.3, axis='y')

# Añadir valores en las barras
for bar, peso in zip(barras, w_final):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + (0.01 if height >= 0 else -0.03),
             f'{peso:.3f}', ha='center', va='bottom' if height >= 0 else 'top')

plt.tight_layout()
plt.show()

print(f"Análisis del entrenamiento:")
print(f"  • El costo disminuyó de {hist_costos[0]:.1f} a {hist_costos[-1]:.1f}")
print(f"  • Reducción: {((hist_costos[0] - hist_costos[-1])/hist_costos[0]*100):.1f}%")

print(f"\nInterpretación de pesos:")
interpretaciones = [
    "Cada pie cuadrado adicional aumenta el precio",
    "Cada habitación adicional aumenta el precio", 
    "Pisos adicionales tienen impacto negativo (¿sorprendente?)",
    "Cada año de edad reduce el precio (casas nuevas valen más)"
]

for feature, peso, interp in zip(feature_names, w_final, interpretaciones):
    signo = "↗️" if peso > 0 else "↘️"
    print(f"  {signo} {feature}: {interp}")

## 5. Evaluación y Predicciones del Modelo

### 5.1 Predicciones en el Conjunto de Entrenamiento

In [ ]:
# Hacer predicciones para todas las casas del entrenamiento
print(f"Predicciones vs Valores Reales:")
print(f"=" * 50)
print(f"{'Casa':<6} {'Tamaño':<8} {'Habitac.':<9} {'Pisos':<6} {'Edad':<5} {'Pred.':<8} {'Real':<8} {'Error':<8}")
print("-" * 65)

errores_absolutos = []
for i in range(len(X_train)):
    # Predicción
    prediccion = np.dot(X_train[i], w_final) + b_final
    error = prediccion - y_train[i]
    error_abs = abs(error)
    errores_absolutos.append(error_abs)
    
    print(f"Casa {i+1:<2d} {X_train[i,0]:<8.0f} {X_train[i,1]:<9.0f} {X_train[i,2]:<6.0f} "
          f"{X_train[i,3]:<5.0f} ${prediccion:<7.0f} ${y_train[i]:<7.0f} ${error:<+7.0f}")

# Estadísticas de error
mae = np.mean(errores_absolutos)  # Error Absoluto Medio
rmse = np.sqrt(np.mean([e**2 for e in [pred - real for pred, real in 
                       zip([np.dot(X_train[i], w_final) + b_final for i in range(len(X_train))], y_train)]]))

print(f"\nEstadísticas de error:")
print(f"  Error Absoluto Medio (MAE): ${mae:.1f}k")
print(f"  Raíz Error Cuadrático Medio (RMSE): ${rmse:.1f}k")
print(f"  Error máximo: ${max(errores_absolutos):.1f}k")

# Calcular R²
predicciones_todas = [np.dot(X_train[i], w_final) + b_final for i in range(len(X_train))]
ss_res = sum([(pred - real)**2 for pred, real in zip(predicciones_todas, y_train)])
ss_tot = sum([(real - np.mean(y_train))**2 for real in y_train])
r2 = 1 - (ss_res / ss_tot)

print(f"  Coeficiente R²: {r2:.3f} ({r2*100:.1f}% de varianza explicada)")

### 5.2 Predicciones para Casas Nuevas

In [ ]:
# Predicciones para casas hipotéticas
casas_nuevas = [
    ([1200, 3, 1, 15], "Casa familiar típica"),
    ([2500, 4, 2, 5], "Casa grande y nueva"),
    ([800, 2, 1, 30], "Casa pequeña, algo vieja"),
    ([3500, 5, 2, 2], "Mansión nueva"),
    ([1800, 3, 2, 20], "Casa mediana"),
]

print(f"Predicciones para Casas Nuevas:")
print(f"=" * 50)
print(f"{'Descripción':<20} {'Features':<20} {'Precio Predicho':<15}")
print("-" * 60)

for features, descripcion in casas_nuevas:
    x_nueva = np.array(features)
    precio_predicho = np.dot(x_nueva, w_final) + b_final
    
    features_str = f"{features[0]}sqft,{features[1]}bed,{features[2]}fl,{features[3]}yrs"
    print(f"{descripcion:<20} {features_str:<20} ${precio_predicho:<14,.0f}")

print(f"\nAnálisis detallado de una predicción:")
x_ejemplo = np.array([1200, 3, 1, 15])
precio = np.dot(x_ejemplo, w_final) + b_final

print(f"Casa: 1200 sqft, 3 habitaciones, 1 piso, 15 años")
print(f"Desglose del cálculo:")
total_features = 0
for i, (feature, peso, valor) in enumerate(zip(feature_names, w_final, x_ejemplo)):
    contribucion = peso * valor
    total_features += contribucion
    print(f"  {feature}: {peso:.4f} × {valor} = ${contribucion:+.0f}")
    
print(f"  Bias: {b_final:+.0f}")
print(f"  Total: ${precio:.0f}")

# Análisis de sensibilidad
print(f"\nAnálisis de sensibilidad (cambio en precio por unidad):")
cambios_unidad = [1, 1, 1, 1]  # 1 sqft, 1 habitación, 1 piso, 1 año
for feature, peso, cambio in zip(feature_names, w_final, cambios_unidad):
    impacto = peso * cambio
    print(f"  +{cambio} {feature}: ${impacto:+.0f}")

### 5.3 Visualización de Predicciones vs Realidad

In [ ]:
# Crear visualizaciones de evaluación
predicciones_entrenamiento = [np.dot(X_train[i], w_final) + b_final for i in range(len(X_train))]
residuos = [pred - real for pred, real in zip(predicciones_entrenamiento, y_train)]

plt.figure(figsize=(15, 5))

# Subplot 1: Predicciones vs Valores Reales
plt.subplot(1, 3, 1)
plt.scatter(predicciones_entrenamiento, y_train, alpha=0.7, s=50)

# Línea diagonal perfecta
min_val = min(min(predicciones_entrenamiento), min(y_train))
max_val = max(max(predicciones_entrenamiento), max(y_train))
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, 
         label='Predicción perfecta')

plt.title('Predicciones vs Valores Reales')
plt.xlabel('Predicciones ($k)')
plt.ylabel('Valores Reales ($k)')
plt.legend()
plt.grid(True, alpha=0.3)

# Subplot 2: Residuos
plt.subplot(1, 3, 2)
plt.scatter(predicciones_entrenamiento, residuos, alpha=0.7, s=50)
plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
plt.title('Residuos vs Predicciones')
plt.xlabel('Predicciones ($k)')
plt.ylabel('Residuos ($k)')
plt.grid(True, alpha=0.3)

# Subplot 3: Histograma de residuos
plt.subplot(1, 3, 3)
plt.hist(residuos, bins=8, alpha=0.7, edgecolor='black')
plt.axvline(x=0, color='red', linestyle='--', linewidth=2)
plt.title('Distribución de Residuos')
plt.xlabel('Residuos ($k)')
plt.ylabel('Frecuencia')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Análisis de residuos:")
print(f"  Media de residuos: ${np.mean(residuos):.1f}k (debería ser ~0)")
print(f"  Desviación estándar: ${np.std(residuos):.1f}k")
print(f"  Residuo máximo (absoluto): ${max([abs(r) for r in residuos]):.1f}k")

# Análisis de features más importantes
importancia_relativa = [abs(w) for w in w_final]
feature_mas_importante = feature_names[np.argmax(importancia_relativa)]
print(f"\nFeature más importante: {feature_mas_importante}")
print(f"Ranking de importancia (por magnitud de peso):")
indices_ordenados = np.argsort(importancia_relativa)[::-1]
for i, idx in enumerate(indices_ordenados):
    print(f"  {i+1}. {feature_names[idx]}: |{w_final[idx]:.4f}|")

## 6. Ejercicios Prácticos

### Ejercicio 1: Analizar el Impacto de Features

In [ ]:
# Ejercicio 1: Analizar qué pasaría si modificamos una feature

def analizar_impacto_feature(casa_base, indice_feature, cambios, w, b):
    """
    Analiza cómo cambia el precio al modificar una feature específica
    
    Args:
        casa_base: Features de la casa base
        indice_feature: Índice de la feature a modificar
        cambios: Lista de cambios a aplicar
        w, b: Parámetros del modelo
    """
    casa_base = np.array(casa_base)
    precio_base = np.dot(casa_base, w) + b
    
    print(f"Análisis de impacto: {feature_names[indice_feature]}")
    print(f"Casa base: {casa_base} → ${precio_base:.0f}k")
    print(f"\n{'Cambio':<10} {'Nueva Feature':<15} {'Nuevo Precio':<12} {'Diferencia':<12}")
    print("-" * 50)
    
    for cambio in cambios:
        casa_modificada = casa_base.copy()
        casa_modificada[indice_feature] += cambio
        precio_nuevo = np.dot(casa_modificada, w) + b
        diferencia = precio_nuevo - precio_base
        
        print(f"{cambio:+<10} {casa_modificada[indice_feature]:<15.0f} "
              f"${precio_nuevo:<11.0f} ${diferencia:+<11.0f}")

# Casa ejemplo: casa mediana
casa_ejemplo = [1500, 3, 1, 20]  # 1500 sqft, 3 habitaciones, 1 piso, 20 años

print("Ejercicio 1: Análisis de Impacto de Features")
print("=" * 50)

# Analizar impacto del tamaño
cambios_tamaño = [-500, -200, 200, 500, 1000]
analizar_impacto_feature(casa_ejemplo, 0, cambios_tamaño, w_final, b_final)

print(f"\n" + "="*50)

# Analizar impacto de habitaciones
cambios_habitaciones = [-1, 1, 2]
analizar_impacto_feature(casa_ejemplo, 1, cambios_habitaciones, w_final, b_final)

print(f"\nConclusión: El tamaño tiene el mayor impacto en el precio")
print(f"Cada 100 sqft adicionales → ${w_final[0]*100:+.0f}k en precio")

### Ejercicio 2: Comparar Modelos con Diferentes Features

In [ ]:
# Ejercicio 2: Entrenar modelos usando solo subconjuntos de features

def entrenar_modelo_subset(X, y, indices_features, nombre_modelo):
    """
    Entrena modelo usando solo un subconjunto de features
    """
    X_subset = X[:, indices_features]
    n_features = X_subset.shape[1]
    
    # Parámetros iniciales
    w_init = np.zeros(n_features)
    b_init = 0.
    
    # Entrenar (versión simplificada, menos iteraciones)
    w, b, _ = gradient_descent_multivariable(
        X_subset, y, w_init, b_init,
        calcular_costo_multivariable, calcular_gradientes_multivariable,
        5.0e-7, 300  # Menos iteraciones para el ejercicio
    )
    
    # Calcular métricas
    predicciones = [np.dot(X_subset[i], w) + b for i in range(len(X_subset))]
    mse = np.mean([(pred - real)**2 for pred, real in zip(predicciones, y)])
    mae = np.mean([abs(pred - real) for pred, real in zip(predicciones, y)])
    
    return {
        'nombre': nombre_modelo,
        'features_usadas': [feature_names[i] for i in indices_features],
        'w': w,
        'b': b,
        'mse': mse,
        'mae': mae,
        'predicciones': predicciones
    }

print("Ejercicio 2: Comparación de Modelos con Diferentes Features")
print("=" * 65)

# Definir diferentes combinaciones de features
modelos_a_probar = [
    ([0], "Solo Tamaño"),
    ([0, 1], "Tamaño + Habitaciones"),
    ([0, 3], "Tamaño + Edad"),
    ([0, 1, 3], "Tamaño + Habitaciones + Edad"),
    ([0, 1, 2, 3], "Todas las Features")
]

resultados = []

for indices, nombre in modelos_a_probar:
    print(f"\nEntrenando: {nombre}")
    resultado = entrenar_modelo_subset(X_train, y_train, indices, nombre)
    resultados.append(resultado)

# Comparar resultados
print(f"\n\nComparación de Modelos:")
print(f"{'Modelo':<25} {'MSE':<8} {'MAE':<8} {'Features':<30}")
print("-" * 75)

for resultado in resultados:
    features_str = ", ".join([f.split()[0] for f in resultado['features_usadas']])
    print(f"{resultado['nombre']:<25} {resultado['mse']:<8.1f} {resultado['mae']:<8.1f} {features_str:<30}")

# Encontrar el mejor modelo
mejor_modelo = min(resultados, key=lambda x: x['mse'])
print(f"\nMejor modelo por MSE: {mejor_modelo['nombre']}")
print(f"Features: {', '.join(mejor_modelo['features_usadas'])}")

# Análisis de complejidad vs rendimiento
print(f"\nAnálisis:")
print(f"  • Usar solo tamaño da resultados razonables")
print(f"  • Añadir más features mejora el modelo hasta cierto punto")
print(f"  • El modelo completo tiene el mejor rendimiento")
print(f"  • Cada feature adicional aporta información valiosa")

## Resumen y Conceptos Clave

### ✅ Lo que has aprendido:

#### 1. **Extensión a Múltiples Variables**:
   - **Modelo**: $f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b$
   - **Vectorización**: Uso eficiente de NumPy para operaciones matriciales
   - **Múltiples features**: Tamaño, habitaciones, pisos, edad

#### 2. **Implementación Vectorizada**:
   - **Predicción**: `np.dot(x, w) + b`
   - **Speedup**: 100x-1000x más rápido que loops
   - **Escalabilidad**: Maneja datasets grandes eficientemente

#### 3. **Función de Costo Multivariable**:
   - **Fórmula**: Misma estructura, pero con vectores
   - **Implementación**: Loop sobre ejemplos, producto punto por predicción
   - **Interpretación**: Mide ajuste global del modelo

#### 4. **Gradient Descent Multivariable**:
   - **Gradientes**: Un gradiente por cada parámetro w_j
   - **Actualización simultánea**: Todos los parámetros se actualizan juntos
   - **Convergencia**: Encuentra mínimo global en función convexa

#### 5. **Análisis de Features**:
   - **Importancia relativa**: Magnitud de pesos indica importancia
   - **Interpretación**: Cada peso representa impacto marginal
   - **Sensibilidad**: Análisis de cambios en features individuales

### 🏠 Aplicación Práctica - Precios de Casas:
- **Modelo final**: 4 features → predicción de precio
- **Rendimiento**: R² = {r2:.3f} (explicación de varianza)
- **Feature más importante**: Tamaño de la casa
- **Aplicación**: Valoración automática de propiedades

### 🚀 Próximos pasos:
- **Feature Scaling**: Normalización para mejor convergencia
- **Learning Rate**: Optimización de hiperparámetros
- **Regularización**: Control de overfitting
- **Feature Engineering**: Creación de features más informativas

### 💡 Puntos clave para recordar:
1. **Vectorización es esencial** para eficiencia en ML
2. **Más features ≠ siempre mejor modelo** (curse of dimensionality)
3. **Interpretabilidad**: Los pesos tienen significado business
4. **Evaluación múltiple**: Usar varias métricas (MSE, MAE, R²)
5. **Análisis de residuos**: Validar supuestos del modelo

### ⚠️ Limitaciones observadas:
- **Learning rate pequeño**: Datos no normalizados requieren α muy bajo
- **Convergencia lenta**: Diferentes escalas de features afectan velocidad
- **Interpretación cuidadosa**: Peso negativo de "pisos" requiere análisis

**Siguiente paso**: Aprenderemos técnicas de feature scaling para resolver estos problemas y mejorar significativamente el entrenamiento.